In [3]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Idiomatic Programmer Code Labs

## Code Labs #3 - Get Familiar with Wide Convolutional Neural Networks

## Prerequistes:

    1. Familiar with Python
    2. Completed Handbook 3/Part 3: Wide Convolutional Neural Networks

## Objectives:

    1. Branch Convolutions in a Inception v1 Module
    2. Branch Convolutions in a ResNeXt Module

## Inception Module as Function API

Let's create an Inception Module.

We will use some elementary approaches:

    1. Dimensionality Reduction replacing one convolution in a pair with a bottleneck convolution.
    2. Branching the input through multiple convolutions (wide).
    3. Concatenating the branches back together.

In [ ]:
from keras import Model, Input
from keras import layers

# Our hypothetical input to an inception module
x = inputs = Input((229, 229, 3))

# The inception branches (where x is the previous layer)
x1 = layers.MaxPooling2D((3, 3), strides=(1,1), padding='same')(x)
# Add the bottleneck after the 2x2 pooling layer
# Hint: x1 is the branch for pooling + bottleneck. So the output from pooling is the input to the bottleneck
x1 = layers.Conv2D(64, ??, strides=??, padding='same')(??)

# Add the second branch which is a single bottleneck convolution
x2 = layers.Conv2D(64, (1, 1), strides=(1, 1), padding='same')(x)  # passes straight through

x3 = layers.Conv2D(64, (1, 1), strides=(1, 1), padding='same')(x)
# Add the the 3x3 convolutional layer after the bottleneck
# Hint: x3 is the branch for bottleneck + convolution. So the output from bottleneck is the input to the convolution
x3 = layers.Conv2D(96, ??, strides=(1, 1), padding='same')(??)

x4 = layers.Conv2D(64, (1, 1), strides=(1, 1), padding='same')(x)
# Add the the 5x5 convolutional layer after the bottleneck
# Hint: x4 is the branch for bottleneck + convolution. So the output from bottleneck is the input to the convolution
x4 = layers.Conv2D(48, ??, strides=(1, 1), padding='same')(??)

# Concatenate the filters from each of the four branches
# Hint: List the branches (variable names) as a list
x = outputs = layers.concatenate([??, ??, ??, ??])

# Let's create a mini-inception neural network using a single inception v1 module
model = Model(inputs, outputs)
model.summary()

### Verify the model architecture using summary method

It should look like below:

```
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
==================================================================================================
input_1 (InputLayer)            (None, 229, 229, 3)  0                                            
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 229, 229, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 229, 229, 64) 256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 229, 229, 64) 256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 229, 229, 64) 256         max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 229, 229, 64) 256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 229, 229, 96) 55392       conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 229, 229, 48) 76848       conv2d_5[0][0]                   
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 229, 229, 272 0           conv2d_1[0][0]                   
                                                                 conv2d_2[0][0]                   
                                                                 conv2d_4[0][0]                   
                                                                 conv2d_6[0][0]                   
==================================================================================================
Total params: 133,264
Trainable params: 133,264
Non-trainable params: 0
```

In [ ]:
model.summary()

## ResNeXt Module as Function API

Let's create an ResNeXt Module.

We will use some elementary approaches:

    1. XX
    2. Split and branching the input through parallel convolutions (wide).
    3. Concatenating the branches back together.

In [4]:
from keras import Input, Model
from keras import layers

def _resnext_block(shortcut, filters_in, filters_out, cardinality=32):
    """ Construct a ResNeXT block
        shortcut   : previous layer and shortcut for identity link
        filters_in : number of filters  (channels) at the input convolution
        filters_out: number of filters (channels) at the output convolution
        cardinality: width of cardinality layer
    """

    # Bottleneck layer
    x = layers.Conv2D(filters_in, kernel_size=(1, 1), strides=(1, 1),
                      padding='same')(shortcut)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Cardinality (Wide) Layer
    filters_card = filters_in // cardinality
    groups = []
    for i in range(cardinality):
        group = layers.Lambda(lambda z: z[:, :, :, i * filters_card:i *
                              filters_card + filters_card])(x)
        groups.append(layers.Conv2D(filters_card, kernel_size=(3, 3),
                                    strides=(1, 1), padding='same')(group))

    # Concatenate the outputs of the cardinality layer together
    x = layers.concatenate(groups)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Bottleneck layer
    x = layers.Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1),
                      padding='same')(x)
    x = layers.BatchNormalization()(x)

    # special case for first resnext block
    if shortcut.shape[-1] != filters_out:
        # use convolutional layer to double the input size to the block so it
        # matches the output size (so we can add them)
        shortcut = layers.Conv2D(filters_out, kernel_size=(1, 1), strides=(1, 1),
                                 padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # Identity Link: Add the shortcut (input) to the output of the block
    x = layers.add([shortcut, x])
    x = layers.ReLU()(x)
    return x

# The input tensor
inputs = layers.Input(shape=(224, 224, 3))

# Stem Convolutional layer
x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

# First ResNeXt Group
for _ in range(3):
    x = _resnext_block(x, 128, 256)

# strided convolution to match the number of output filters on next block and
# reduce by 2
x = layers.Conv2D(512, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)

# Second ResNeXt Group
for _ in range(4):
    x = _resnext_block(x, 256, 512)

# strided convolution to match the number of output filters on next block and
# reduce by 2
x = layers.Conv2D(1024, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)

# Third ResNeXt Group
for _ in range(6):
    x = _resnext_block(x, 512, 1024)

# strided convolution to match the number of output filters on next block and
# reduce by 2
x = layers.Conv2D(2048, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)

# Fourth ResNeXt Group
for _ in range(3):
    x = _resnext_block(x, 1024, 2048)

# Final Dense Outputting Layer for 1000 outputs
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1000, activation='softmax')(x)

model = Model(inputs, outputs)


In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 112, 112, 64) 256         conv2d_36[0][0]                  
__________________________________________________________________________________________________
re_lu_4 (ReLU)                  (None, 112, 112, 64) 0           batch_normalization_5[0][0]      
__________________________________________________________________________________________________
max_poolin

lambda_76 (Lambda)              (None, 56, 56, 4)    0           re_lu_8[0][0]                    
__________________________________________________________________________________________________
lambda_77 (Lambda)              (None, 56, 56, 4)    0           re_lu_8[0][0]                    
__________________________________________________________________________________________________
lambda_78 (Lambda)              (None, 56, 56, 4)    0           re_lu_8[0][0]                    
__________________________________________________________________________________________________
lambda_79 (Lambda)              (None, 56, 56, 4)    0           re_lu_8[0][0]                    
__________________________________________________________________________________________________
lambda_80 (Lambda)              (None, 56, 56, 4)    0           re_lu_8[0][0]                    
__________________________________________________________________________________________________
lambda_81 

conv2d_150 (Conv2D)             (None, 28, 28, 8)    584         lambda_137[0][0]                 
__________________________________________________________________________________________________
conv2d_151 (Conv2D)             (None, 28, 28, 8)    584         lambda_138[0][0]                 
__________________________________________________________________________________________________
conv2d_152 (Conv2D)             (None, 28, 28, 8)    584         lambda_139[0][0]                 
__________________________________________________________________________________________________
conv2d_153 (Conv2D)             (None, 28, 28, 8)    584         lambda_140[0][0]                 
__________________________________________________________________________________________________
conv2d_154 (Conv2D)             (None, 28, 28, 8)    584         lambda_141[0][0]                 
__________________________________________________________________________________________________
conv2d_155

__________________________________________________________________________________________________
lambda_204 (Lambda)             (None, 28, 28, 8)    0           re_lu_20[0][0]                   
__________________________________________________________________________________________________
lambda_205 (Lambda)             (None, 28, 28, 8)    0           re_lu_20[0][0]                   
__________________________________________________________________________________________________
lambda_206 (Lambda)             (None, 28, 28, 8)    0           re_lu_20[0][0]                   
__________________________________________________________________________________________________
lambda_207 (Lambda)             (None, 28, 28, 8)    0           re_lu_20[0][0]                   
__________________________________________________________________________________________________
lambda_208 (Lambda)             (None, 28, 28, 8)    0           re_lu_20[0][0]                   
__________

lambda_287 (Lambda)             (None, 14, 14, 16)   0           re_lu_26[0][0]                   
__________________________________________________________________________________________________
lambda_288 (Lambda)             (None, 14, 14, 16)   0           re_lu_26[0][0]                   
__________________________________________________________________________________________________
conv2d_279 (Conv2D)             (None, 14, 14, 16)   2320        lambda_257[0][0]                 
__________________________________________________________________________________________________
conv2d_280 (Conv2D)             (None, 14, 14, 16)   2320        lambda_258[0][0]                 
__________________________________________________________________________________________________
conv2d_281 (Conv2D)             (None, 14, 14, 16)   2320        lambda_259[0][0]                 
__________________________________________________________________________________________________
conv2d_282

                                                                 conv2d_309[0][0]                 
                                                                 conv2d_310[0][0]                 
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 14, 14, 512)  2048        concatenate_9[0][0]              
__________________________________________________________________________________________________
re_lu_27 (ReLU)                 (None, 14, 14, 512)  0           batch_normalization_29[0][0]     
__________________________________________________________________________________________________
conv2d_311 (Conv2D)             (None, 14, 14, 1024) 525312      re_lu_27[0][0]                   
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, 14, 14, 1024) 4096        conv2d_311[0][0]                 
__________

lambda_382 (Lambda)             (None, 14, 14, 16)   0           re_lu_35[0][0]                   
__________________________________________________________________________________________________
lambda_383 (Lambda)             (None, 14, 14, 16)   0           re_lu_35[0][0]                   
__________________________________________________________________________________________________
lambda_384 (Lambda)             (None, 14, 14, 16)   0           re_lu_35[0][0]                   
__________________________________________________________________________________________________
conv2d_381 (Conv2D)             (None, 14, 14, 16)   2320        lambda_353[0][0]                 
__________________________________________________________________________________________________
conv2d_382 (Conv2D)             (None, 14, 14, 16)   2320        lambda_354[0][0]                 
__________________________________________________________________________________________________
conv2d_383

conv2d_482 (Conv2D)             (None, 7, 7, 2048)   2099200     re_lu_43[0][0]                   
__________________________________________________________________________________________________
conv2d_483 (Conv2D)             (None, 7, 7, 1024)   2098176     conv2d_482[0][0]                 
__________________________________________________________________________________________________
batch_normalization_46 (BatchNo (None, 7, 7, 1024)   4096        conv2d_483[0][0]                 
__________________________________________________________________________________________________
re_lu_44 (ReLU)                 (None, 7, 7, 1024)   0           batch_normalization_46[0][0]     
__________________________________________________________________________________________________
lambda_449 (Lambda)             (None, 7, 7, 32)     0           re_lu_44[0][0]                   
__________________________________________________________________________________________________
lambda_450

                                                                 conv2d_562[0][0]                 
                                                                 conv2d_563[0][0]                 
                                                                 conv2d_564[0][0]                 
                                                                 conv2d_565[0][0]                 
                                                                 conv2d_566[0][0]                 
                                                                 conv2d_567[0][0]                 
                                                                 conv2d_568[0][0]                 
                                                                 conv2d_569[0][0]                 
                                                                 conv2d_570[0][0]                 
                                                                 conv2d_571[0][0]                 
          